In [1]:
import os
from dotenv import load_dotenv
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

# Load environment variables
load_dotenv('server/.env.dev')
hf_api_token = os.getenv('HUGGINGFACEHUB_API_TOKEN')

def load_llm_from_hf():
    llm = HuggingFaceEndpoint(
        repo_id="HuggingFaceH4/zephyr-7b-beta",
        task="text-generation",
        max_new_tokens=512,
        do_sample=False,
        repetition_penalty=1.03,
        api_key=hf_api_token
    )

    chat_model = ChatHuggingFace(llm=llm)
    return chat_model

def respond(survey):
    response_schemas = [
        ResponseSchema(
            name="Goal",
            description="The user's gym goal."
        ),
        ResponseSchema(
            name="FreeTime",
            description="The amount of free time the user has each week."
        ),
        ResponseSchema(
            name="ClassSchedule",
            description="The user's weekly class schedule."
        ),
        ResponseSchema(
            name="Weight",
            description="The user's current weight."
        ),
        ResponseSchema(
            name="Height",
            description="The user's height."
        ),
    ]

    output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
    format_instructions = output_parser.get_format_instructions()

    prompt = PromptTemplate(
        template="Generate a workout schedule for the next 7 days considering the user's goals, free time, class schedule, weight, and height.\n{format_instructions}\n{user_data}",
        input_variables=["user_data"],
        partial_variables={"format_instructions": format_instructions},
    )

    llm = load_llm_from_hf()
    chain = prompt | llm | output_parser

    # Prepare user data to be passed into the chain
    user_data = {
        "Goal": survey['What is the gym goal'],
        "FreeTime": survey['How much free time you have'],
        "ClassSchedule": survey['What is your weekly schedule looks like'],
        "Weight": survey['What is your current weight?'],
        "Height": survey['What is your height?'],
    }

    try:
        response = chain.invoke({"user_data": user_data})
    except Exception as e:
        print(f"An error occurred: {e}")
        response = {"error": str(e)}

    return response

if __name__ == "__main__":
    survey = {
        "What is the gym goal": "Lose weight",
        "How much free time you have": "10 hours per week",
        "What is your weekly schedule looks like": {
            "Monday": "9:00 AM - 10:30 AM (Math), 2:00 PM - 3:30 PM (History)",
            "Tuesday": "10:00 AM - 11:30 AM (Physics), 1:00 PM - 2:30 PM (Chemistry)",
            "Wednesday": "9:00 AM - 10:30 AM (Math), 2:00 PM - 3:30 PM (History)",
            "Thursday": "10:00 AM - 11:30 AM (Physics), 1:00 PM - 2:30 PM (Chemistry)",
            "Friday": "9:00 AM - 10:30 AM (English), 2:00 PM - 3:30 PM (Biology)",
        },
        "What is your current weight?": "70 kg",
        "What is your height?": "170 cm",
        "What is your preferred workout environment?": "Gym",
        "Are there any specific exercises or activities you enjoy?": "Running, Swimming, Yoga",
        "Do you have any injuries or physical limitations?": "Knee pain when running, lower back issues",
        "What is your current fitness level?": "Intermediate",
        "What is your desired intensity level for workouts?": "Medium"
    }

    print(respond(survey))


WARNING! api_key is not default parameter.
                    api_key was transferred to model_kwargs.
                    Please make sure that api_key is what you intended.
/Users/sohanurrahman/Desktop/Projects/fitness_app/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


An error occurred: Got invalid return object. Expected key `Weight` to be present, but got {'Goal': 'Lose weight', 'FreeTime': '10 hours per week', 'ClassSchedule': {'Monday': '9:00 AM - 10:30 AM (Math), 2:00 PM - 3:30 PM (History)', 'Tuesday': '10:00 AM - 11:30 AM (Physics'}}
{'error': "Got invalid return object. Expected key `Weight` to be present, but got {'Goal': 'Lose weight', 'FreeTime': '10 hours per week', 'ClassSchedule': {'Monday': '9:00 AM - 10:30 AM (Math), 2:00 PM - 3:30 PM (History)', 'Tuesday': '10:00 AM - 11:30 AM (Physics'}}"}
